In [ ]:
!pip install rouge.score nltk py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install accelerate==0.20.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk, load_metric
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
max_input = 80
max_target = 80
batch_size = 128
model_checkpoints = "google/pegasus-x-base"

In [ ]:
data = load_dataset("bogdancazan/news-not-not-ela-text-simplification")
data

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Normal', 'Simple'],
        num_rows: 102711
    })
    test: Dataset({
        features: ['Normal', 'Simple'],
        num_rows: 1414
    })
})

In [ ]:
train_data = data['train'].to_pandas()
train_data

,Normal,Simple
0,poll finds americans ok with women in combat,women get the ok to fight in combat units publ...
1,poll finds americans ok with women in combat,women get the ok to fight in combat units
2,women get the ok to fight in combat units publ...,women get the ok to fight in combat units
3,wars of the future will be fought with america...,american women will soon be free to fight on t...
4,wars of the future will be fought with america...,american women will soon be able to fight on t...
...,...,...
102706,its kernels are more spherical than other corn...,the thin layer or hull that surrounds the seed...
102707,its kernels are more spherical than other corn...,there is a thin layer or hull that covers the ...
102708,the thin layer or hull that surrounds the seed...,there is a thin layer or hull that covers the ...
102709,the thin layer or hull that surrounds the seed...,there is a thin layer that covers the seed.


In [ ]:
test_data = data['test'].to_pandas()
test_data

,Normal,Simple
0,when the kernel is heated above degrees celsiu...,when the kernel gets very hot and the water in...
1,when the kernel is heated above degrees celsiu...,when the kernel gets very hot above degrees ce...
2,when the kernel is heated above degrees celsiu...,when the kernel gets very hot above degrees fa...
3,when the kernel gets very hot and the water in...,when the kernel gets very hot above degrees fa...
4,when the kernel gets very hot above degrees ce...,when the kernel gets very hot above degrees fa...
...,...,...
1409,this week u.s. ambassador to colombia kevin wh...,this week u.s. ambassador to colombia kevin wh...
1410,and the country is on track to produce tons th...,colombian land dedicated to avocado production...
1411,colombian land dedicated to avocado production...,the country is on track to produce tons this y...
1412,but tons is n t enough he said looking forwar...,but tons is n t enough iragorri said looking ...


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments
tokenizer = PegasusTokenizer.from_pretrained(model_checkpoints)

In [ ]:
def preprocess_data(data_to_process):
  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, max_length=max_input, padding='max_length', truncation=True, pad_to_max_length=True, return_tensors="pt")
    decodings = tokenizer(labels, max_length=max_input, padding='max_length', truncation=True, pad_to_max_length=True, return_tensors="pt")
    encodings['labels'] = decodings['input_ids']
    return  encodings

  return  tokenize_data(data_to_process['Normal'], data_to_process['Simple'])

In [ ]:
tokenize_train_data = data['train'].map(preprocess_data, batched = True)
tokenize_train_data

Dataset({
    features: ['Normal', 'Simple', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 102711
})

In [ ]:
tokenize_test_data = data['test'].map(preprocess_data, batched = True)
tokenize_test_data

Dataset({
    features: ['Normal', 'Simple', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1414
})

In [ ]:
import torch
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model =PegasusForConditionalGeneration.from_pretrained(model_checkpoints).to(torch_device)

You are using a model of type pegasus_x to instantiate a model of type pegasus. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at google/pegasus-x-base were not used when initializing PegasusForConditionalGeneration: ['model.encoder.layers.11.global_self_attn_layer_norm.weight', 'model.encoder.layers.9.global_self_attn_layer_norm.weight', 'model.encoder.layers.7.global_self_attn_layer_norm.bias', 'model.encoder.layers.9.global_self_attn_layer_norm.bias', 'model.encoder.layers.2.global_self_attn_layer_norm.bias', 'model.encoder.layers.5.global_self_attn_layer_norm.bias', 'model.encoder.layers.7.global_self_attn_layer_norm.weight', 'model.encoder.layers.0.global_self_attn_layer_norm.bias', 'model.encoder.layers.1.global_self_attn_layer_norm.bias', 'model.encoder.layers.5.global_self_attn_layer_norm.weight', 'model.encoder.layers.3.global_self_attn_layer_norm.bias', 'model.encoder.embed_global.weight', 'model.encoder.layer

In [ ]:
metric = load_metric('rouge')

<ipython-input-17-10e4a614ebd6>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [ ]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [ ]:
args = Seq2SeqTrainingArguments(
    'pegasus-text-simplification_1e4_adafactor_wikilarge_20epici', #save directory
    evaluation_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size= 128,
    # gradient_accumulation_steps=4,
    # weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=20,
    predict_with_generate=True,
    # eval_accumulation_steps=3,
    fp16=True, #available only with CUDA
    optim="adafactor",
    push_to_hub=True,
    )

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenize_train_data,
    eval_dataset=tokenize_test_data,
    data_collator=collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_rouge
)

/content/pegasus-text-simplification_1e4_adafactor_wikilarge_20epici is already a clone of https://huggingface.co/bogdancazan/pegasus-text-simplification_1e4_adafactor_wikilarge_20epici. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.954200,0.341560
2,0.311100,0.337202
3,0.291900,0.335631
4,0.265900,0.338885
5,0.247600,0.342077
6,0.235100,0.347379
7,0.221500,0.349617
8,0.214100,0.354791
9,0.201500,0.360715
10,0.192100,0.362833


Several commits (2) will be pushed upstream.


TrainOutput(global_step=16060, training_loss=0.22788886418229765, metrics={'train_runtime': 8628.4979, 'train_samples_per_second': 238.074, 'train_steps_per_second': 1.861, 'total_flos': 1.957052251865088e+17, 'train_loss': 0.22788886418229765, 'epoch': 20.0})

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/1.11G [00:00<?, ?B/s]

Upload file runs/Jun25_15-19-56_6b17d9e0ea56/events.out.tfevents.1687706399.6b17d9e0ea56.14072.0:   0%|       …

To https://huggingface.co/bogdancazan/pegasus-text-simplification_1e4_adafactor_wikilarge_20epici
   393a952..e9345fc  main -> main

   393a952..e9345fc  main -> main

To https://huggingface.co/bogdancazan/pegasus-text-simplification_1e4_adafactor_wikilarge_20epici
   e9345fc..774c5af  main -> main

   e9345fc..774c5af  main -> main



'https://huggingface.co/bogdancazan/pegasus-text-simplification_1e4_adafactor_wikilarge_20epici/commit/e9345fc929b2ec0922f78f108b4462b1ed5b85c2'

In [ ]:
!git clone https://github.com/feralvam/easse.git && cd easse && pip install .

fatal: destination path 'easse' already exists and is not an empty directory.


In [ ]:
!cd easse && pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/easse
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-ha77xnyw/tseval_1cbc6a7586794f1dab3a46b5cda0b020
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-ha77xnyw/tseval_1cbc6a7586794f1dab3a46b5cda0b020
  Resolved https://github.com/facebookresearch/text-simplification-evaluation.git to commit f335e2e27026321c7c3d1dd63857416c7e7397b2
  Preparing metadata (setup.py) ... done
  Using cached sacrebleu-2.3.1-py3-none-any.whl (118 kB)
  Using cached sacremoses-0.0.53.tar.gz (880 kB)
  Preparing metadata (setup.py) ... done
  Using cached stanfordnlp-0.2.0-py3-none-any.whl (158 kB)
  Using cached yattag-1.15.1.tar.gz (28 kB)
  Preparing metadata (setup.py) ... done
  Using 

In [ ]:
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pandas as pd
from easse.sari import corpus_sari
from sacrebleu import corpus_bleu
import textstat

def evall(dataset, id):
  predictions = []
  actuals = []
  ref = []

  for _, sentence in dataset.iterrows():
    model_inputs = tokenizer(sentence['Normal'],  max_length=max_input, padding='max_length', truncation=True, pad_to_max_length=True, return_tensors="pt")

    generated_ids = model.generate(
              input_ids = model_inputs['input_ids'].to("cuda"),
              attention_mask = model_inputs['attention_mask'].to("cuda"),
              max_length=80,
              num_beams=4,
              repetition_penalty=2.5,
              length_penalty=1.0,
              early_stopping=True
              )

    rez = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    predictions.extend(rez)
    actuals.append(sentence['Normal'])
    ref.append(sentence['Simple'])

  final_df = pd.DataFrame({"Actual Text": actuals, "Reff Text": ref, "Generated Text": predictions})
  final_df.to_csv(os.path.join("outputs", "predictions_" + str(id) + ".csv"))

  eval = pd.DataFrame()
  eval['sari']=[corpus_sari(orig_sents=final_df['Actual Text'],
            sys_sents=final_df['Generated Text'],
            refs_sents=[final_df['Reff Text']])]
  eval['blue']=corpus_bleu(final_df['Generated Text'].tolist(), [final_df['Reff Text'].tolist()]).score
  count = 0
  for index, row in final_df.iterrows():
      if row['Generated Text'] == row['Actual Text']:
        count+=1

  eval['dublicates'] = count
  fkgl_scores = [textstat.flesch_kincaid_grade(text) for text in final_df['Generated Text']]
  eval["fkgl"] = sum(fkgl_scores) / len(fkgl_scores)

  eval.to_csv(os.path.join("outputs", "eval_" + str(id) + ".csv"))


In [ ]:
 evall(data['test'].to_pandas(), 0)